### 주식 데이터 수집(fdr, 스크래핑 등)

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [2]:
import pickle
import time
import FinanceDataReader as fdr
import io
from tqdm import tqdm
from datetime import datetime

In [3]:
from pymongo import MongoClient
from bson.objectid import ObjectId
client = MongoClient('127.0.0.1')
db = client.stock

In [4]:
import seaborn as sns

In [5]:
# def se(df,start,end):
#     df = df[(df.index>=start)*(df.index<=end)].reset_index()
    
#     return(df)

### KOSPI / KOSDAQ의 현재 상장된 종목 코드

In [6]:
class MarketCodes():
    def __init__(self,market):
        if(market!='KOSPI')&(market!='KOSDAQ'):
            print("Error : Please input KOSPI or KOSDAQ")
        self.market=market
            
    def getCodes(self):
        '''
        market = 'KOSPI' or 'KOSDAQ'
        '''
        kospiDf = fdr.StockListing(self.market)
        kospiDf = kospiDf[kospiDf.apply(lambda x:len(x['Symbol'])==6,axis=1)] #이상한 몇호 뭐시기 이런거 들어있어서 제외
        codes = kospiDf['Symbol'].tolist()
        return codes

    def closedJongmok(self):
        '''
        code = [kospi : STK, kosdaq : KSQ]
        '''
        if(self.market=='KOSPI'):
            code = 'STK'
        else:
            code = 'KSQ'
        url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
                    'name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&' \
                    'market_gubun='+code+'&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&' \
                    'isu_srt_cd=&fromdate=20000101&todate=22001231&del_cd=1&' \
                    'pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'

        header_data = {
            'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
        }
        r = requests.get(url, headers=header_data)

        # STEP 02: download
        url = 'http://file.krx.co.kr/download.jspx'
        form_data = {'code': r.text}
        header_data = {
            'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
            'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
        }
        r = requests.post(url, data=form_data, headers=header_data)
        df = pd.read_excel(io.BytesIO(r.content))
        df['종목코드'] = df['종목코드'].str.replace('A', '')
        df['폐지일'] = pd.to_datetime(df['폐지일'])
        col_map = {'종목코드':'Symbol', '기업명':'Name', '폐지일':'DelistingDate', '폐지사유':'Reason'}
        df=df.rename(columns = col_map)
        codes = df['Symbol'].tolist()
        return(codes)
    
    def allCodes(self):
        '''
        상장폐지종목 포함 코드
        '''
        codes = self.getCodes() + self.closedJongmok()
        return codes

In [7]:
market='KOSPI'
codeC = MarketCodes("KOSPI")
codes = codeC.getCodes() #현재 상장주식만
#codes = codeC.allCodes() #상장폐지 종목 포함

### 일일 주가 데이터

In [9]:
#Mongodb에 저장
def storeDailyPrice(codes, market,year):
    today = datetime.today().strftime('%Y-%m-%d') #저장당시 날짜
    
    for idx,code in tqdm(enumerate(codes)):
        element = {}
        element['code'] = code
        element['date'] = today
        element['market'] = market
        element['price'] = fdr.DataReader(code, year).reset_index().to_dict('list')
        db.dailyPrice.insert_one(element)

In [101]:
storeDailyPrice(codes, market, '2000')

916it [03:00,  5.07it/s]


In [22]:
#db.dailyPrice.find_one()

### 재무제표 크롤링
sql이용해 저장해야될듯 - 나중에

In [23]:
#ref : https://engkimbs.tistory.com/625
url = "https://finance.naver.com/item/coinfo.nhn?code=005930&target=finsum_more"
url = "https://finance.naver.com/item/coinfo.nhn?code=005930"
url = "https://finance.naver.com/item/main.nhn?code=005930"

In [24]:
soup = bs(requests.get(url).text,'lxml')

In [25]:
table = soup.find("div",{"class":"section cop_analysis"})
table = table.find('div',{"class":"sub_section"})

In [10]:
jaemu = []

In [11]:
dump=[]

In [48]:
def getFsNaver(code):
    URL = "https://finance.naver.com/item/main.nhn?code="+code

    samsung_electronic = requests.get(URL)
    html = samsung_electronic.text

    soup = bs(html, 'lxml')
    finance_html = soup.select('div.section.cop_analysis div.sub_section')[0]

    th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
    annual_date = th_data[3:7]
    quarter_date = th_data[7:13]
    
    finance_index = [item.get_text().strip() for item in finance_html.select('th.h_th2')][3:]
    finance_data = [item.get_text().strip() for item in finance_html.select('td')]
    finance_data = np.array(finance_data)
    finance_data.resize(len(finance_index), 10)
    finance_date = annual_date + quarter_date
    finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index, columns=finance_date)
    return finance

In [ ]:
for idx,code in enumerate(codes):

    URL = "https://finance.naver.com/item/main.nhn?code="+code

    samsung_electronic = requests.get(URL)
    html = samsung_electronic.text

    soup = bs(html, 'lxml')
    try:
        finance_html = soup.select('div.section.cop_analysis div.sub_section')[0]
    
    except IndexError:
        print('error : ',code)
        dump.append(code)
        time.sleep(10)
        continue


    th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
    annual_date = th_data[3:7]
    quarter_date = th_data[7:13]
    
    finance_index = [item.get_text().strip() for item in finance_html.select('th.h_th2')][3:]
    finance_data = [item.get_text().strip() for item in finance_html.select('td')]
    finance_data = np.array(finance_data)
    finance_data.resize(len(finance_index), 10)
    finance_date = annual_date + quarter_date
    
    try:
        finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index, columns=finance_date)
    except:
        print('error2 : ',code)
        dump.append(code)
        continue

    jaemu.append({'code':code,'df':finance})
    
    if(idx%10==9):
        print(idx/len(codes))
        time.sleep(5)
    #time.sleep(np.random.rand(1)[0]*0.8)


In [254]:
with open('jaemu_0511.pkl','wb') as f:
    pickle.dump(jaemu,f)

In [255]:
with open('dump.pkl_0511','wb') as f:
    pickle.dump(dump,f)

In [ ]:
codes_ = list(x for x in codes if x not in dump)

In [ ]:
#KB손해보험 주식 1주당 0.57287주의 KB금융지주 주식으로 교환할 수 있게 됨
#지주회사의 완전자사화 등 -> 상폐x

### 거래주체(기관/외인)

In [182]:
from urllib import parse

In [194]:
parse.unquote('%EC%82%BC%EC%84%B1%EC%A4%91%EA%B3%B5%EC%97%85')

'삼성중공업'

In [ ]:
%EC%A0%84%EC%B2%B4

In [51]:
def tradeWho(code):
    url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
                'name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&' \
                'market_gubun='+code+'&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&' \
                'isu_srt_cd=&fromdate=20000101&todate=22001231&del_cd=1&' \
                'pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'

    header_data = {
        'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.get(url, headers=header_data)
    
    url = 'http://file.krx.co.kr/download.jspx'
    form_data = {'code': r.text}
    header_data = {
        'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
        'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.post(url, data=form_data, headers=header_data)
    df = pd.read_excel(io.BytesIO(r.content))
    df['종목코드'] = df['종목코드'].str.replace('A', '')
    df['폐지일'] = pd.to_datetime(df['폐지일'])
    col_map = {'종목코드':'Symbol', '기업명':'Name', '폐지일':'DelistingDate', '폐지사유':'Reason'}
    df=df.rename(columns = col_map)
    return df

In [99]:
'&isu_cdnm=A010140%2F%EC%82%BC%EC%84%B1%EC%A4%91%EA%B3%B5%EC%97%85'\


'&isu_cdnm=A010140%2F%EC%82%BC%EC%84%B1%EC%A4%91%EA%B3%B5%EC%97%85&isu_cd=KR7010140002&isu_nm=%EC%82%BC%EC%84%B1%EC%A4%91%EA%B3%B5%EC%97%85&'

### 우선주면 100, 보통주면 000

In [ ]:
'&isu_cd=KR7010140002'\

In [145]:
temp

'name=fileDown&filetype=xls&url=MKD/13/1302/13020304/mkd13020304_01&isu_cd=KR7010140002&isu_nm=%EC%82%BC%EC%84%B1%EC%A4%91%EA%B3%B5%EC%97%85&type=D&period_selector=day&fromdate=20201114&todate=20201214&pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020304%2FMKD13020304.jsp'

In [142]:
temp='name=fileDown&filetype=xls&url=MKD/13/1302/13020304/mkd13020304_01&isu_cdnm=A039490%2F%ED%82%A4%EC%9B%80%EC%A6%9D%EA%B6%8C&isu_cd=KR7039490008&isu_nm=%ED%82%A4%EC%9B%80%EC%A6%9D%EA%B6%8C&isu_srt_cd=A039490&type=D&period_selector=day&fromdate=20201114&todate=20201214&pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020304%2FMKD13020304.jsp'

In [291]:
def getFullCode(code):
    checkNum = 0
    for idx, i in enumerate(code):
        if(idx%2==0):
            checkNum+=int(i)
        else:
            temp = int(i)*2
            if(temp>=10):
                temp = 1+temp%10
            checkNum +=temp
#    checkNum += 20#KR
    checkNum = checkNum%10
    if(checkNum!=0):
        checkNum = 10-checkNum
    code = f'KR7{code}00{checkNum}'
    return code

In [299]:
code = codes[2]
fullCode = getFullCode(code)
fromDate = '20150101'

In [311]:
def tradeWho(code, fromDate):
    if(code[-1] !='0'):
        print('not valid code form, pleas enter xxxxx0')
        return -1
    fullCode = getFullCode(code)
    url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?'\
    'name=fileDown&filetype=xls&url=MKD/13/1302/13020304/mkd13020304_01'\
    '&isu_cd='+fullCode+\
    '&isu_nm=%EC%A0%84%EC%B2%B4&'\
    'type=D&period_selector=day'\
    '&fromdate='+fromDate+'&todate=20201214&pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020304%2FMKD13020304.jsp'
    
    header_data = {
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.get(url, headers=header_data)
    
    
    form_data = {'code': r.text}
    url = 'http://file.krx.co.kr/download.jspx'

    header_data = {
            'Referer': 'http://marketdata.krx.co.kr/contents/MKD/13/1302/13020304/MKD13020304.jsp',
            'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
        }
    r = requests.post(url, data=form_data, headers=header_data)

    return pd.read_excel(io.BytesIO(r.content))

In [313]:
tradeWho('005930', '20190101')

,년/월/일,종가,대비,거래량(주),기관_매수량(주),기관_매도량(주),기관_순매수(주),외국인_매수량(주),외국인_매도량(주),외국인_순매수(주)
0,2020/12/14,"73,800",400,"21,510,415","9,283,590","11,360,355","-2,076,765","4,456,764","4,586,276","-129,512"
1,2020/12/11,"73,400",500,"18,486,672","9,706,086","9,683,384","22,702","2,506,264","4,520,060","-2,013,796"
2,2020/12/10,"72,900","-1,000","39,882,928","20,421,820","21,612,579","-1,190,759","5,420,538","13,633,239","-8,212,701"
3,2020/12/09,"73,900","2,200","24,077,996","12,832,947","12,020,584","812,363","4,569,069","4,906,812","-337,743"
4,2020/12/08,"71,700","-1,200","25,000,338","11,565,954","12,376,152","-810,198","3,206,542","6,691,453","-3,484,911"
...,...,...,...,...,...,...,...,...,...,...
479,2019/01/07,"38,750","1,300","12,748,997","3,715,768","3,986,021","-270,253","6,253,620","4,534,644","1,718,976"
480,2019/01/04,"37,450",-150,"14,108,958","5,528,599","4,632,695","895,904","5,120,455","6,638,796","-1,518,341"
481,2019/01/03,"37,600","-1,150","12,471,493","4,256,279","6,107,514","-1,851,235","3,931,426","3,670,607","260,819"
482,2019/01/02,"38,750",50,"7,847,664","3,302,583","3,847,780","-545,197","2,891,113","1,607,308","1,283,805"


### 업종별 per
이거도 sql한다음에 다시분석해보자

industry별로 list 만들어서 유사도 검정 or 몇개 테마만 골라

In [6]:
kospiDf = pd.read_csv("./data/upjong_0623.csv")

In [85]:
kospiDf=kospiDf.dropna()

In [87]:
kospiDf[kospiDf['Name']=='셀트리온헬스케어']

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1104,091990,KOSDAQ,셀트리온헬스케어,기타 전문 도매업,바이오의약품 마케팅 및 판매,2017-07-28,12월,김형기,http://www.celltrionhealthcare.com/kr/index.do,인천광역시


In [92]:
kospiDf[kospiDf.apply(lambda x:'자동차' in x['Industry'],axis=1)]

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
30,005830,KOSPI,DB손해보험,보험업,"자동차보험,화재보험,해상보험,특종보험,장기보험",1973-06-28,12월,김정남,http://www.idbins.com,서울특별시
48,130500,KOSDAQ,GH신소재,기타 섬유제품 제조업,"자동차용 부직포, 폴리우레탄 폼",2013-08-02,12월,우희구,http://www.gumho-nt.com,경상북도
62,089470,KOSPI,HDC현대EP,기타 화학제품 제조업,"합성수지(PP컴파운딩가공,자동차범퍼,내장제) 제조,도소매",2006-09-25,12월,김 명 호(직무대행 선임),http://www.hyundai-ep.com,충청남도
64,011200,KOSPI,HMM,해상 운송업,"정기,부정기,전용선,콘테이너선,자동차선,LNG선 해운",1995-10-05,12월,배재훈,http://www.hmm21.com,서울특별시
97,009070,KOSPI,KCTC,도로 화물 운송업,"화물자동차운송,항만하역,창고보관,컨테이너조작,중량물해상운송",1978-09-29,12월,이준환,http://www.kctc.co.kr,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2492,001450,KOSPI,현대해상,보험업,손해보험(자동차보험),1989-08-25,12월,"조용일, 이성재",http://www.hi.co.kr,서울특별시
2506,013520,KOSPI,화승알앤에이,자동차 신품 부품 제조업,"자동차용 고무제품(제조저압호스,고압호스,산업용고무호스,,WEATHER STIRP,고...",1991-02-22,12월,"현지호,이정두",http://www.hsrna.com,경상남도
2509,010690,KOSPI,화신,자동차 신품 부품 제조업,"자동차엔진부품(ARM류,C/MBR류),자동차부품 제조",1994-01-14,12월,"정서진, 장의호",http://www.hwashin.co.kr,경상북도
2510,126640,KOSDAQ,화신정공,자동차 신품 부품 제조업,자동차부품,2010-08-31,12월,정 서 진,http://hsp.hwashin.co.kr,경상북도


In [11]:
kospi = kospiDf[kospiDf['Market']=='KOSPI']

In [79]:
soft = kospiDf[kospiDf['Sector']=='특수 목적용 기계 제조업']
soft=soft.dropna()

In [22]:
jejo = kospiDf.dropna()[kospiDf.dropna().apply(lambda x:'자동차'  in x['Sector'], axis=1)]

In [24]:
kospiDf.groupby('Sector').size().reset_index().sort_values(0,ascending = False).head(30)


,Sector,0
141,특수 목적용 기계 제조업,137
78,소프트웨어 개발 및 공급업,129
125,전자부품 제조업,120
111,자동차 신품 부품 제조업,97
104,의약품 제조업,91
34,기타 금융업,88
47,기타 화학제품 제조업,86
25,금융 지원 서비스업,77
140,통신 및 방송 장비 제조업,73
57,반도체 제조업,57


In [178]:
code = ([x for x in jejo if x in codeTemp])

In [179]:
df = perTimeDf[code].mean().reset_index()

df.columns = ['code','per']

In [180]:
reluMean(df['per'])

/Users/edward/opt/anaconda3/envs/edward_park/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/edward/opt/anaconda3/envs/edward_park/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


(27.216100100284958, 0.15753424657534246)

In [114]:
def upjong(name):

    codeTemp = perTimeDf.columns.tolist()
    code = kospiDf[kospiDf['Sector'] == name]['Symbol'].tolist()

    code = ([x for x in code if x in codeTemp])

    df = perTimeDf[code].mean().reset_index()

    df.columns = ['code','per']
    return(df)

In [112]:
def reluMean(arr):
    arr=np.array(arr)
    return (np.mean(arr[arr>0]), len(arr[arr<0])/len(arr))

In [152]:
reluMean(upjong('기타 식품 제조업')['per'].tolist())

(36.94565108362261, 0.0)

In [130]:
temp = perTimeDf['272210'][:5].values

In [148]:
upjong('금융 지원 서비스업')

,code,per
0,016610,41.169215
1,030210,8.765298
2,005940,12.188581
3,001510,18.105961
4,030610,4.814256
5,003540,9.720416
6,008560,7.515140
7,006800,11.514433
8,001270,8.678502
9,016360,11.128393


In [121]:
reluMean(upjong('전자부품 제조업')['per'].tolist())

/Users/edward/opt/anaconda3/envs/edward_park/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/edward/opt/anaconda3/envs/edward_park/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


(27.68780637126737, 0.21052631578947367)

In [99]:
arr = medicalDf['per'].tolist()

### 2018년 per 

In [42]:
timePriceDf = pd.read_csv("kospi18_0519.csv", index_col = 0)

In [52]:
#per, pbr, eps 등 지수와 코드 가져오는 함수
def indexYear(index, year):
    codes = []
    lists = []

    for i in range(len(jaemu)):
        
        try: 
            jipyo = jaemu[i]['df'].loc[index, year]

            if(type(jipyo) == str):
                if(jipyo==''):
                    continue

                lists.append(jipyo)
                codes.append(jaemu[i]['code'])

            elif(jipyo.tolist()[0]!=''):
                lists.append(jipyo.tolist()[0])
                codes.append(jaemu[i]['code'])



        except KeyError:
            continue
    return(codes, lists)

In [45]:
with open('jaemu_0511.pkl','rb') as f:
    jaemu = pickle.load(f)

In [53]:
indexDf = pd.DataFrame({'code':indexYear('EPS(원)','2017.12')[0],
                        'index_2017':indexYear('EPS(원)','2017.12')[1]})

In [55]:
indexDf['index_2017'] = indexDf['index_2017'].str.replace(',','')
indexDf['index_2017'] = indexDf['index_2017'].astype(float)
indexDf = indexDf[indexDf['index_2017'] !=0]

In [57]:
perTimeDf = timePriceDf[indexDf['code']]/indexDf['index_2017'].tolist() #per 10기준으로 판단

In [61]:
perTimeDf = perTimeDf[perTimeDf.index<'2019']

In [62]:
perTimeDf

,095570,006840,027410,282330,138930,001460,001040,079160,000120,011150,...,004800,094280,093370,081660,005870,079980,005010,069260,000540,003280
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,21.781250,9.784632,0.206771,21.727911,7.623282,41.197994,12.800569,-1145.454545,70.243902,22.755906,...,8.872721,16.840731,32.131661,16.174757,8.974359,16.178737,29.362101,8.756254,4.056532,-1.563559
2018-01-03,21.625000,9.532039,0.206104,20.902147,7.526273,40.791435,12.909399,-1074.242424,70.731707,22.559055,...,9.415273,18.798956,32.601881,16.990291,9.014423,16.409861,28.705441,8.756254,4.033613,-1.583686
2018-01-04,22.093750,9.625100,0.200768,20.695706,7.502021,40.723675,12.764344,-1084.848485,70.731707,22.362205,...,9.064179,17.558747,31.661442,16.582524,8.814103,16.486903,28.799250,8.613295,3.964859,-1.555085
2018-01-05,22.656250,9.625100,0.194765,20.644096,7.502021,40.655915,12.655591,-1092.424242,69.024390,22.401575,...,9.128034,17.428198,31.818182,16.737864,8.733974,16.255778,29.174484,8.756254,4.094729,-1.567797
2018-01-08,22.281250,9.638394,0.194765,20.540875,7.663703,40.655915,12.583064,-1083.333333,67.804878,22.007874,...,8.745011,17.885117,31.065831,16.407767,8.453526,16.024653,28.799250,9.042173,4.094729,-1.555085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,13.546875,7.351768,0.114458,22.502064,6.200485,33.066811,9.899631,-650.000000,83.170732,18.937008,...,6.363931,14.229765,23.636364,48.932039,6.314103,11.479199,22.420263,9.274482,3.422460,-0.859110
2018-12-24,13.484375,7.218825,0.111657,21.727911,6.200485,33.202331,9.718274,-640.909091,81.463415,18.543307,...,6.439435,13.838120,22.978056,47.572816,6.474359,11.679507,22.514071,9.399571,3.453018,-0.843220
2018-12-26,13.468750,7.125764,0.108855,21.676301,6.143897,33.676650,9.536994,-612.878788,80.487805,17.834646,...,6.417862,13.577023,22.319749,47.572816,6.498397,11.587057,21.857411,9.506791,3.414820,-0.809322


In [ ]:
#아이투자

# url = 'http://www.itooza.com/vclub/y10_page.php?cmp_cd=005930&mode=dy&ss=10&sv=2&lsmode=1&lkmode=1&pmode=1&exmode=1&accmode=1'

# source = requests.get(url).content

# soup= bs(source,'lxml')

# soup

# soup.findAll("div",{"class":"body"})

### 상장폐지 데이터

### Trash

In [ ]:
temp = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=csv&url=MKD/13/1302/13020303/mkd13020303&isu_cdnm=A003550%2FLG&isu_cd=KR7003550001&isu_nm=LG&isu_srt_cd=A003550&period_selector=day&fromdate=20200801&todate=20200901&pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020303%2FMKD13020303.jsp'


temp

url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
'name=fileDown&filetype=xls&url=MKD/13/1302/13020303/mkd13020303&' \
'isu_cdnm=%2F%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&' \
'isu_cd=KR7003550002&isu_nm=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&' \
'isu_srt_cd=&period_selector=day&fromdate=20200801&todate=20200901&' \
'pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020303%2FMKD13020303.jsp'




len('KR 7 00355 000 2')

url

header_data = {
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
}
r = requests.get(url, headers=header_data)

url = 'http://file.krx.co.kr/download.jspx'
form_data = {'code': r.text}
header_data = {
    'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
}
r = requests.post(url, data=form_data, headers=header_data)
df = pd.read_excel(io.BytesIO(r.content))
df=df.rename(columns = col_map)
